Load in the weather, smartmeter and gasmeter data as 3 seperate dataframes. 
Goal is to combine them into one dataframe.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

df =pd.read_csv("/datc/opschaler/smartmeter_data/P01S01W0373.csv",header=0,delimiter=";",parse_dates=['Timestamp', 'gasTimestamp'])

# Split the smartmeter and gasmeter data
smart = df.iloc[:,:7]
gas = df.iloc[:, 7:]

#Read in weather data
weather_path = "//datc//opschaler//weather_data//20170307to20170329.csv"
weather_column_names = ['STN','YYYYMMDD','HH','DD','FH','FF','FX','T','T10','TD','SQ','Q','DR','RH','P','VV','N','U','WW','IX','M','R','S','O','Y']

weather = pd.read_csv(weather_path, delimiter=',', comment='#', names=weather_column_names, parse_dates=['YYYYMMDD', 'HH'])
weather['T'] = weather['T']*0.01 # Convert temperature to celcius instead of 0.1 celcius
weather['HH'] = weather['HH']+':00:00'
weather['HH'] = weather['HH'].str.strip() # removes the blank spaces infront of the strings
weather['HH'] = weather['HH'].str.replace('24:00:00', '23:59:59') # KNMI uses 24 for 00 hours.... to have to convert this for to_datetime to be able to work
weather.loc[:,'datetime'] = pd.to_datetime(weather['YYYYMMDD'].astype(str)+' '+weather['HH'].astype(str)) #create datetime column

In [2]:
smart.head()

,Timestamp,eMeter,eMeterReturn,eMeterLow,eMeterLowReturn,ePower,ePowerReturn
0,2017-03-07 18:14:28,3717.472,0.0,3883.382,0.0,376.0,0.0
1,2017-03-07 18:14:38,3717.473,0.0,3883.382,0.0,381.0,0.0
2,2017-03-07 18:14:48,3717.474,0.0,3883.382,0.0,378.0,0.0
3,2017-03-07 18:14:59,3717.475,0.0,3883.382,0.0,376.0,0.0
4,2017-03-07 18:15:09,3717.476,0.0,3883.382,0.0,380.0,0.0


In [3]:
gas.head()

,gasTimestamp,gasMeter
0,2017-03-07 18:00:00,1599.792
1,2017-03-07 18:00:00,1599.792
2,2017-03-07 18:00:00,1599.792
3,2017-03-07 18:00:00,1599.792
4,2017-03-07 18:00:00,1599.792


In [4]:
weather.head()

,STN,YYYYMMDD,HH,DD,FH,FF,FX,T,T10,TD,...,N,U,WW,IX,M,R,S,O,Y,datetime
0,344,2018-08-01,1:00:00,0,10,0,10,1.33,,128,...,0,97,,5,0,0,0,0,0,2018-08-01 01:00:00
1,344,2018-08-01,2:00:00,80,0,10,10,1.37,,132,...,1,97,,5,0,0,0,0,0,2018-08-01 02:00:00
2,344,2018-08-01,3:00:00,990,10,10,20,1.33,,128,...,6,97,,5,0,0,0,0,0,2018-08-01 03:00:00
3,344,2018-08-01,4:00:00,340,10,10,20,1.25,,123,...,8,98,10,7,0,0,0,0,0,2018-08-01 04:00:00
4,344,2018-08-01,5:00:00,300,0,10,10,1.37,,132,...,8,97,,5,0,0,0,0,0,2018-08-01 05:00:00
